<a href="https://colab.research.google.com/github/MarioMarkov/cars-predict/blob/master/cars_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold

#Load data
data = pd.read_csv('/content/cars-data.csv')
test = pd.read_csv('/content/test-cars.csv')

#data.price = np.log10(data.price ) 

# Remove Id column
data = data.drop(['id'],axis='columns')

# Remove outliers in IQR 70-20
Q3 = np.quantile(data.price, 0.70)
Q1 = np.quantile(data.price, 0.20)
IQR = Q3 - Q1
lower_range = Q1 - 1.5 * IQR
upper_range = Q3 + 1.5 * IQR
outlier_free_list = [x for x in data.price if (
    (x > lower_range) & (x < upper_range))]
data = data.loc[data.price.isin(outlier_free_list)]



#data.price = np.log10(data.price) 

# Train set without price col
X = data.drop(['price'],axis='columns')

# Train set price col
y = data.price

# Encoding string columns to numeric
nominal_cols = X.select_dtypes(include='object')
le = preprocessing.LabelEncoder()
for col in nominal_cols:
  X[col] = le.fit_transform(X[col])
  test[col] = le.transform(test[col])

# Space of possible hyperparameters
space = dict()
space['n_estimators'] = range(100,600,100)
space['learning_rate'] = [0.01,0.02,0.03,0.04,0.05,0.1,0.2,0.3,0.4,0.5]

# Defining model to try hyperparameters from space on
model = XGBRegressor(objective='reg:squarederror')

#X_train, X_valid, y_train,y_valid = train_test_split(X,y,random_state=1,test_size=0.2)

# Fit model
model.fit(X,y)

#Calculate error 
mae = -1 * cross_val_score(model, X, y,
                                  cv=3,
                                  scoring='neg_mean_absolute_error')
#preds = pd.DataFrame(model.predict(X_valid))
#mae = mean_absolute_error(preds,y_valid)

print(mae.mean())


#preds.to_csv("preds.csv",index = False)

preds =  model.predict(test)
#preds = 10 ** preds
df = pd.DataFrame({"brand":test.brand,
                     "price":preds})
print(df)
#y_valid = 10 ** y_valid
#y_valid.to_csv("real.csv",index = False)



2865.572296544125
   brand        price
0     27  2009.866821
1      1  7811.062012


In [86]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedKFold
from sklearn.neighbors import LocalOutlierFactor



data = pd.read_csv('/content/cars-data.csv')
data = data[1:]


print(data.price.describe())


Q3 = np.quantile(data.price, 0.75)
Q1 = np.quantile(data.price, 0.25)
IQR = Q3 - Q1
print("IQR value for column %s is: %s" % ('price', IQR))
global outlier_free_list
global filtered_data
lower_range = Q1 - 1.5 * IQR
upper_range = Q3 + 1.5 * IQR
outlier_free_list = [x for x in data.price if (
    (x > lower_range) & (x < upper_range))]
filtered_data = data.loc[data.price.isin(outlier_free_list)]
 
print(filtered_data.price.describe())


count       995.000000
mean      13369.934673
std       12928.062509
min         650.000000
25%        5600.000000
50%        9499.000000
75%       16500.000000
max      144444.000000
Name: price, dtype: float64
IQR value for column price is: 10900.0
count      928.000000
mean     10841.085129
std       7128.286736
min        650.000000
25%       5500.000000
50%       8850.000000
75%      14500.000000
max      32500.000000
Name: price, dtype: float64
